In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import uuid
import logging
import getpass
import numpy as np
import cv2
import os
import gc
import time
import imageio
import random
import shutil
import copy
import glob
import json
import shutil
import pickle
import traceback
import zipfile
from collections import Counter
from pprint import pprint as print

In [3]:
import cv2
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm, tqdm_notebook
from PIL import Image, ImageDraw, ExifTags
from shapely.geometry import Polygon
import imgaug
import imgaug as ia
import imgaug.augmenters as iaa
import skimage
from skimage import io, color
import torch
from torch.utils.data import DataLoader

In [4]:
def imshow(img, figsize=(16, 16)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    ax.imshow(img)
    
def remove_all_content(fd):
    shutil.rmtree(fd)
    os.mkdir(fd)

In [5]:
from test_api import CLI, CVAT_API_V1, ResourceType

In [ ]:
class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("", "")  # type tuple
        self.server_host = ""
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = "test"
        self.labels = [{"name":"text", "attributes": []}]  # type list
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = ["/home/phan.huy.hoang/Pictures/test_img_cvat.zip"]
        self.annotation_path = "/home/phan.huy.hoang/cvat/utils/cli/instances_default.json"
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

args = Args()
CVAT_URL = "http://{}:{}/".format(args.server_host, args.server_port)
API_URL = CVAT_URL + "api/v1/"
username, password = args.auth
username, password

In [7]:
def get_cookies():
    login_json = {"username": username, "password": password}
    login_req = requests.post(API_URL + 'auth/login', json=login_json)
    cookies = login_req.cookies
    return cookies

cookies = get_cookies()

In [ ]:
req = requests.get("{}/api/v1/tasks/1043".format(CVAT_URL), cookies=cookies)
req.text

In [9]:
task_ids = list(range(1043, 1047)) + list(range(1048, 1063))  # ignore task 1047
print(len(task_ids))

19


### Download cả ảnh + label

In [10]:
saved_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/downloaded_cropper/zip"
assert os.path.exists(saved_dir)

In [11]:
url_fmt = "http://{}:8080/api/v1/tasks/{}/dataset?format=COCO%201.0&action=download"

In [12]:
# remove_all_content(saved_dir)

In [13]:
def download_file(req, task_id):
    with open(os.path.join(saved_dir, '{}.zip'.format(task_id)), 'wb') as f:
        for chunk in req.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("Task {}: download completed".format(task_id))

In [14]:
for task_id in tqdm_notebook(task_ids):
    url = url_fmt.format(args.server_host, task_id)

    while True:
        req = requests.get(url, cookies=cookies)
        if req.status_code == 200:
            download_file(req, task_id)
            break

    time.sleep(10)
    
# unzip all zip files in the same root folder
for task_id in task_ids:
    zip_fp = os.path.join(saved_dir, "{}.zip".format(task_id))
    assert os.path.exists(zip_fp)
    
    task_dir = os.path.join(saved_dir, str(task_id))
    if not os.path.exists(task_dir):
        os.mkdir(task_dir)
        
    with zipfile.ZipFile(zip_fp, "r") as zip_ref:
        zip_ref.extractall(task_dir)

/home/phan.huy.hoang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/19 [00:00<?, ?it/s]

'Task 1043: download completed'
'Task 1044: download completed'
'Task 1045: download completed'
'Task 1046: download completed'
'Task 1048: download completed'
'Task 1049: download completed'
'Task 1050: download completed'
'Task 1051: download completed'
'Task 1052: download completed'
'Task 1053: download completed'
'Task 1054: download completed'
'Task 1055: download completed'
'Task 1056: download completed'
'Task 1057: download completed'
'Task 1058: download completed'
'Task 1059: download completed'
'Task 1060: download completed'
'Task 1061: download completed'
'Task 1062: download completed'


### Cắt ảnh theo perspective transform (4 góc)

In [15]:
def get_center_point(c):
    x, y, w, h = c
    x_center = int(x + w / 2)
    y_center = int(y + h / 2)
    return x_center, y_center

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [16]:
warped_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/downloaded_cropper/warped"
assert os.path.exists(warped_dir)

In [17]:
remove_all_content(warped_dir)

In [18]:
def check_existed_or_create(fd):
    if not os.path.exists(fd):
        os.mkdir(fd)
        
for type_idcard in ["cmnd", "cccd", "cccd_chip"]:
    first_dir = os.path.join(warped_dir, type_idcard)
    check_existed_or_create(first_dir)
    for mode in ["front", "back"]:
        second_dir = os.path.join(first_dir, mode)
        check_existed_or_create(second_dir)

In [19]:
error_fps = []

def process_task(anno_fp, img_dir):
    # for 1 task
    with open(anno_fp, "r") as f:
        json_data = json.load(f)
    
    json_df = pd.DataFrame(json_data["annotations"])
    json_df = pd.concat([json_df.drop(['attributes'], axis=1), json_df['attributes'].apply(pd.Series)], axis=1)
    
    label_mapping = dict()
    for d in json_data["categories"]:
        label_mapping[d["name"]] = d["id"]
    
    for img_index, img_info in enumerate(tqdm_notebook(json_data["images"])):
        img_id = img_info["id"]
        img_fn = img_info["file_name"]
        img_fp = os.path.join(img_dir, img_fn)

        try:
            img_df = json_df[json_df["image_id"].isin([img_id])]
            pil_img = Image.open(img_fp)
            pil_img = pil_img.convert('RGB')
            img = np.array(pil_img)

            if len(img_df) == 4:
                is_front = False
            else:
                is_front = True
            # check nb rows == 4 or 5 (4/5 classes)
            # check 4 or 5 unique category ids (ignore wrong annotation)
            # idcard's type it not np.nan (annotated in top-left object)
            try:
                assert len(img_df) in [4, 5]
                if is_front:
                    assert len(img_df["category_id"].unique()) == 5
                else:
                    assert len(img_df["category_id"].unique()) == 4
                assert img_df[img_df["category_id"] == 1]["type"].values[0] is not np.nan
            except Exception:
    #             print("Error: {}".format(img_fp))
                error_fps.append(img_fp)
                continue

            is_valid = True
            for row_index, row in img_df.iterrows():
                c_id = row["category_id"]
                bbox = copy.deepcopy(row["bbox"])
                if c_id == label_mapping["top_left"]:
                    tl = bbox
                    type_idcard = row["type"]
                    if type_idcard is np.nan:
                        is_valid = False
                        break
                elif c_id == label_mapping["top_right"]:
                    tr = bbox
                elif c_id == label_mapping["bottom_left"]:
                    bl = bbox
                elif c_id == label_mapping["bottom_right"]:
                    br = bbox
            if not is_valid:
                continue

            if type_idcard not in ["cmnd", "cccd", "cccd_chip"]:
                error_fps.append(img_fp)
                continue

            pts = np.array([
                get_center_point(tl),
                get_center_point(tr),
                get_center_point(br),
                get_center_point(bl)
            ])
            pts = order_points(pts)
            warped_img = four_point_transform(img, pts)

            if is_front:
                sub_dir = "front"
            else:
                sub_dir = "back"
            warped_img_fp = os.path.join(warped_dir, type_idcard, sub_dir, "warped_{}".format(img_fn))

            # rotate image
            h, w, _ = warped_img.shape
            if h > w:
                # check rotated 90 degree image
                x1, y1 = get_center_point(tl)
                x3, y3 = get_center_point(br)
                if x1 > x3:
                    # rotate -90 degree
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                else:
                    # rotate 90 degree
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_90_CLOCKWISE)
            else:
                # check reverse image
                y_center_top = (get_center_point(tl)[1] + get_center_point(tr)[1]) / 2 
                y_center_bottom = (get_center_point(bl)[1] + get_center_point(br)[1]) / 2 
                if y_center_top > y_center_bottom:
                    warped_img = cv2.rotate(warped_img, cv2.ROTATE_180)

            del tl, tr, br, bl
        #     print("{} - {}".format(img_fn, is_convert_rgb))
#             print("{} - {}".format(img.shape, warped_img.shape))
            imageio.imwrite(warped_img_fp, warped_img)
        except Exception:
            error_fps.append(img_fp)
            traceback.print_exc()
            continue

In [ ]:
for task_id in tqdm_notebook(task_ids):
    task_dir = os.path.join(saved_dir, str(task_id))
    img_dir = os.path.join(task_dir, "images")
    anno_fp = os.path.join(task_dir, "annotations", "instances_default.json")
    assert os.path.exists(task_dir)
    assert os.path.exists(anno_fp)
    assert os.path.exists(img_dir)
    
    process_task(anno_fp, img_dir)

In [21]:
len(error_fps)

47

### Upload data to server

cmnd
- front
- back

cccd
- front
- back

In [22]:
CCCD_LABEL_NAMES = [
    'gender',
    'expires_date',
    'nationality',
    'ethnicity',
    'id_number',
    'name',
    'dob',
    'hometown',
    'address',
    'face',
    'religion',
    'issue_date',
    'personal_identification',
    'left_fingerprint',
    'right_fingerprint',
    'remove'
]

CMND_LABEL_NAMES = [
    'ethnicity',
    'id_number',
    'name',
    'dob',
    'hometown',
    'address',
    'face',
    'religion',
    'issue_date',
    'personal_identification',
    'left_fingerprint',
    'right_fingerprint',
    'remove'
]
print(len(CCCD_LABEL_NAMES))
print(len(CMND_LABEL_NAMES))

16
13


In [23]:
CCCD_LABELS = []
CMND_LABELS = []

for label in CCCD_LABEL_NAMES:
    CCCD_LABELS.append({
        "name": label,
        "attributes": [
            {
                "name": "text_label",
                "input_type": "text",
                "mutable": False,
                "values": [
                    ""
                ]
            } 
        ]
    })
    
for label in CMND_LABEL_NAMES:
    CMND_LABELS.append({
        "name": label,
        "attributes": [
            {
                "name": "text_label",
                "input_type": "text",
                "mutable": False,
                "values": [
                    ""
                ]
            } 
        ]
    })

In [24]:
from test_api import CLI, CVAT_API_V1, ResourceType

class Args:
    
    def __init__(self, action="create"):
        
        self.action = action
        self.auth = ("", "")  # type tuple
        self.server_host = ""
        self.server_port = 8080
        self.https = False
        self.loglevel = 20

        self.name = ""
#         self.labels = LABELS
        self.labels = []
        self.overlap = 0
        self.segment_size = 0
        self.bug = ""
        self.resource_type = ResourceType(0)
        self.resources = [""]
        self.annotation_path = ""
        self.annotation_format = "COCO 1.0"

        # for git only
        self.completion_verification_period = 10
        self.dataset_repository_url = ""
        self.lfs = False

args = Args()

actions = {
    'create': CLI.tasks_create,
    'delete': CLI.tasks_delete,
    'ls': CLI.tasks_list,
    'frames': CLI.tasks_frame,
    'dump': CLI.tasks_dump,
    'upload': CLI.tasks_upload
}

In [25]:
root_dir = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/downloaded_cropper/warped"

In [26]:
cmnd_dir = os.path.join(root_dir, "cmnd")
cccd_dir = os.path.join(root_dir, "cccd")
cmnd_front_dir = os.path.join(cmnd_dir, "front")
cmnd_back_dir = os.path.join(cmnd_dir, "back")
cccd_front_dir = os.path.join(cccd_dir, "front")
cccd_back_dir = os.path.join(cccd_dir, "back")

In [27]:
cmnd_front_fps = glob.glob(os.path.join(cmnd_front_dir, "*"))
cmnd_back_fps = glob.glob(os.path.join(cmnd_back_dir, "*"))
cccd_front_fps = glob.glob(os.path.join(cccd_front_dir, "*"))
cccd_back_fps = glob.glob(os.path.join(cccd_back_dir, "*"))

In [28]:
for i in [cmnd_front_fps, cmnd_back_fps, cccd_front_fps, cccd_back_fps]:
    print(len(i))

1268
1144
690
615


In [29]:
save_image_fd = "/media/SUN-ASTERISK\phan.huy.hoang/My Passport/phanhoang/data/CMT_CCCD/CVAT/detector"
assert os.path.exists(save_image_fd)

In [30]:
remove_all_content(save_image_fd)

In [31]:
def create_batches_image(img_fps, type_id="", suffix=""):

    batch_index = 0
    batch_dir_fmt = "{}_{}_{}"
    if suffix == "front":
        max_img_per_batch = 50
    else:
        max_img_per_batch = 100

    for index, img_fp in tqdm_notebook(enumerate(img_fps)):

        saved_batch_dir = os.path.join(save_image_fd, batch_dir_fmt.format(batch_index, type_id, suffix))
        if not os.path.exists(saved_batch_dir):
            os.mkdir(saved_batch_dir)

        img_fn = img_fp.strip("/").split("/")[-1]
        new_img_fn = os.path.join(saved_batch_dir, img_fn)
        shutil.copy(img_fp, new_img_fn)

        if ((index + 1) % max_img_per_batch == 0 and index > 0) or index == len(img_fps) - 1:
            shutil.make_archive(
                saved_batch_dir,
                "zip", saved_batch_dir
            )
            batch_index += 1
            
#         if index == 101:
#             break

    print(batch_index)
    shutil.make_archive(
        saved_batch_dir,
        "zip", saved_batch_dir
    )
    print(">>> Completed!")

In [32]:
create_batches_image(cmnd_front_fps, type_id="cmnd", suffix="front")
create_batches_image(cmnd_back_fps, type_id="cmnd", suffix="back")
create_batches_image(cccd_front_fps, type_id="cccd", suffix="front")
create_batches_image(cccd_back_fps, type_id="cccd", suffix="back")

/home/phan.huy.hoang/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


0it [00:00, ?it/s]

26
'>>> Completed!'


0it [00:00, ?it/s]

12
'>>> Completed!'


0it [00:00, ?it/s]

14
'>>> Completed!'


0it [00:00, ?it/s]

7
'>>> Completed!'


### Upload to CVAT

In [33]:
zip_fps = sorted(glob.glob(os.path.join(save_image_fd, "*.zip")))
print(len(zip_fps))

59


In [34]:
for i, (zip_fp, _) in tqdm(enumerate(zip(zip_fps, zip_fps)), total=len(zip_fps)):

    zip_fn = zip_fp.split("/")[-1]
    *_, type_id, suffix = zip_fn[:-4].split("_")
    
    if type_id == "cmnd":
        args.labels = CMND_LABELS
    elif type_id == "cccd":
        args.labels = CCCD_LABELS
    else:
        raise NotImplementedError()
    
    args.annotation_path = ""
    args.resources = [zip_fp]
    task_name = "[eKYC_DETECTOR]{}_{}_{}".format(int(time.time()), type_id, suffix)
    args.name = task_name

    with requests.Session() as session:
        api = CVAT_API_V1('{}:{}'.format(args.server_host, args.server_port), args.https)
        cli = CLI(session, api, args.auth)

        args.action = "create"
#         task_json, resp_json = actions["create"](cli, **args.__dict__)
        res = actions["create"](cli, **args.__dict__)
        gc.collect()

    time.sleep(10)

  0%|          | 0/59 [00:00<?, ?it/s]

<Response [201]>


  2%|▏         | 1/59 [00:24<23:45, 24.58s/it]

<Response [201]>


  3%|▎         | 2/59 [00:41<18:57, 19.96s/it]

...

 97%|█████████▋| 57/59 [1:08:50<00:25, 12.52s/it]

<Response [201]>


 98%|█████████▊| 58/59 [1:09:04<00:12, 12.90s/it]

<Response [201]>


100%|██████████| 59/59 [1:09:17<00:00, 70.46s/it]


In [35]:
len(zip_fps) / 5

11.8